# Prompt Audit — Systematic Verification

Verifies that every code example in `examples.py` actually works when run through the executor.
Also cross-references column names in personas against actual enriched views.

**Run this after fixing prompts.** All code blocks should execute without errors.

In [1]:
# Setup
import sys
sys.path.insert(0, '../src')

from dotenv import load_dotenv
load_dotenv('../.env')

import re
import pandas as pd
from alfred_fpl.bi.data_access import QuerySpec, Filter, fetch_df, fetch_enriched
from alfred_fpl.bi.analytics import (
    add_rolling_mean, compute_form_trend, compute_differentials,
    compute_fixture_difficulty, compute_price_velocity, rank_by,
)
from alfred_fpl.bi.viz import render_line, render_bar, render_heatmap, render_comparison
from alfred_fpl.bi.executor import execute
from alfred_fpl.domain.prompts.examples import _EXAMPLES
from alfred_fpl.domain.prompts.personas import _PERSONAS

from IPython.display import Image, display

print('Loaded. Ready to audit.')

Loaded. Ready to audit.


## 1. Extract Code Blocks from examples.py

Pull every \`\`\`python block from the examples and catalog them.

In [2]:
# Extract all code blocks from examples
code_blocks = []
for key, text in _EXAMPLES.items():
    blocks = re.findall(r'```python\n(.*?)```', text, re.DOTALL)
    for i, block in enumerate(blocks):
        code_blocks.append({
            'key': key,
            'index': i,
            'code': block.strip(),
        })

print(f'Found {len(code_blocks)} code blocks across {len(_EXAMPLES)} example entries\n')
for cb in code_blocks:
    print(f'  {cb["key"]} [{cb["index"]}]: {cb["code"][:60]}...')

Found 11 code blocks across 12 example entries

  squad:analyze [0]: # Compute formation: merge squad with players for position
s...
  scouting:analyze [0]: df = df_players.copy()
df["pts_per_m"] = df["total_points"] ...
  scouting:analyze [1]: # Use add_rolling_mean helper instead of manual rolling
df =...
  scouting:analyze [2]: # Use compute_fixture_difficulty helper for a specific team
...
  market:analyze [0]: # Filter candidates by position
candidates = df_players[df_p...
  market:analyze [1]: # Use compute_price_velocity helper on snapshot data
velocit...
  league:analyze [0]: # Use compute_differentials helper for squad comparison
diff...
  league:analyze [1]: my_trend = df_league_standings[df_league_standings["manager_...
  live:analyze [0]: # Merge enriched squad with GW stats for current-GW points
#...
  fixtures:analyze [0]: # Enriched fixtures already have team short names (home_team...
  fixtures:analyze [1]: # Build FDR grid (self-contained)
rows = []
for _, fix in df...

## 2. Load Test Data

Fetch real data from Supabase to use as executor context.

In [3]:
# Load data for all executor contexts
df_players = fetch_enriched('players', order_by='total_points', ascending=False, limit=50)
df_fixtures = fetch_enriched('fixtures', order_by='gameweek', limit=100)
df_squads = fetch_enriched('squad', limit=15)
df_league_standings = fetch_df(QuerySpec(table='league_standings', limit=50))

# Player gameweeks for top 2 players
if len(df_players) >= 2:
    top_ids = df_players.head(2)['id'].tolist()
    df_player_gameweeks = fetch_df(QuerySpec(
        table='player_gameweeks',
        filters=[Filter('player_id', 'in', top_ids)],
        order_by='gameweek',
        limit=100,
    ))
else:
    df_player_gameweeks = pd.DataFrame()

# Player snapshots
if len(df_players) >= 1:
    df_player_snapshots = fetch_df(QuerySpec(
        table='player_snapshots',
        filters=[Filter('player_id', 'eq', df_players.iloc[0]['id'])],
        limit=20,
    ))
else:
    df_player_snapshots = pd.DataFrame()

# Build the full context dict (what the executor would have)
test_context = {
    'df_players': df_players,
    'df_fixtures': df_fixtures,
    'df_squads': df_squads,
    'df_league_standings': df_league_standings,
    'df_player_gameweeks': df_player_gameweeks,
    'df_player_snapshots': df_player_snapshots,
}

print('Test data loaded:')
for name, df in test_context.items():
    print(f'  {name}: {df.shape}')

Test data loaded:
  df_players: (50, 21)
  df_fixtures: (100, 13)
  df_squads: (15, 13)
  df_league_standings: (50, 11)
  df_player_gameweeks: (52, 22)
  df_player_snapshots: (4, 10)


## 3. Execute Every Code Block

Run each extracted code block through the executor. Report pass/fail.

In [4]:
# Run every code block through the executor
results = []

# Some code blocks reference variables that would come from prior context.
# Provide reasonable defaults so the executor doesn't fail on missing vars.
extra_context = {}
if not df_fixtures.empty and 'home_team_id' in df_fixtures.columns:
    extra_context['target_team_id'] = df_fixtures.iloc[0]['home_team_id']
if not df_league_standings.empty and 'manager_id' in df_league_standings.columns:
    managers = df_league_standings['manager_id'].unique()
    if len(managers) >= 2:
        extra_context['my_id'] = managers[0]
        extra_context['rival_id'] = managers[1]
# For league differential: need two separate squad DataFrames
if not df_squads.empty:
    half = len(df_squads) // 2
    extra_context['df_squads_mine'] = df_squads.iloc[:max(half, 1)]
    extra_context['df_squads_rival'] = df_squads.iloc[max(half, 1):]

full_context = {**test_context, **extra_context}

for cb in code_blocks:
    result = execute(cb['code'], context=full_context)
    status = 'PASS' if result.error is None else 'FAIL'
    results.append({
        'key': cb['key'],
        'index': cb['index'],
        'status': status,
        'error': result.error,
        'charts': len(result.charts),
        'dfs': list(result.dataframes.keys()),
    })
    print(f'[{status}] {cb["key"]}[{cb["index"]}]', end='')
    if result.charts:
        print(f' [{len(result.charts)} charts]', end='')
    if result.error:
        print(f'\n  ERROR: {result.error[:120]}')
    else:
        print()

passed = sum(1 for r in results if r['status'] == 'PASS')
failed = sum(1 for r in results if r['status'] == 'FAIL')
print(f'\nResults: {passed}/{len(results)} passed, {failed} failed')

[PASS] squad:analyze[0]
[PASS] scouting:analyze[0]
[PASS] scouting:analyze[1]
[PASS] scouting:analyze[2]
[PASS] market:analyze[0]
[PASS] market:analyze[1]
[PASS] league:analyze[0]
[PASS] league:analyze[1] [1 charts]
[PASS] live:analyze[0]
[PASS] fixtures:analyze[0]
[PASS] fixtures:analyze[1] [1 charts]

Results: 11/11 passed, 0 failed


## 4. Column Reference Audit

Check what column names appear in persona text and verify they exist in enriched views.

In [5]:
# Audit column references in personas
# Check that column names mentioned in analyze personas exist in the relevant DataFrames

# Known columns per enriched view
view_columns = {}
for name, df in test_context.items():
    if not df.empty:
        view_columns[name] = set(df.columns)

print('Available columns per DataFrame:')
for name, cols in view_columns.items():
    print(f'  {name}: {sorted(cols)[:10]}...' if len(cols) > 10 else f'  {name}: {sorted(cols)}')

# Extract column-like references from analyze personas
print('\n--- Persona Column References ---')
for subdomain, personas in _PERSONAS.items():
    analyze = personas.get('analyze', '')
    # Find potential column references (words that look like column names)
    # These are words after "df_" or in common patterns
    df_refs = re.findall(r'df_(\w+)', analyze)
    col_patterns = re.findall(r'["\']([\w_]+)["\']', analyze)
    if df_refs or col_patterns:
        print(f'\n{subdomain}:analyze')
        if df_refs:
            print(f'  DataFrame refs: {df_refs}')
        if col_patterns:
            print(f'  Column-like refs: {col_patterns}')

Available columns per DataFrame:
  df_players: ['assists', 'bonus', 'clean_sheets', 'first_name', 'form', 'fpl_id', 'goals_scored', 'id', 'minutes', 'news']...
  df_fixtures: ['away_difficulty', 'away_score', 'away_team', 'away_team_id', 'finished', 'fpl_id', 'gameweek', 'home_difficulty', 'home_score', 'home_team']...
  df_squads: ['gameweek', 'id', 'is_captain', 'is_vice_captain', 'manager_id', 'manager_name', 'multiplier', 'player_form', 'player_id', 'player_name']...
  df_league_standings: ['event_points', 'gameweek', 'id', 'last_rank', 'league_id', 'league_name', 'manager_id', 'manager_name', 'rank', 'team_name']...
  df_player_gameweeks: ['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded', 'gameweek']...
  df_player_snapshots: ['form', 'gameweek', 'id', 'player_id', 'points_per_game', 'price', 'selected_by_percent', 'snapshot_time', 'transfers_in_event', 'transfers_out_event']

---

## 5. Prompt Logging Setup (for live Alfred pipeline)

When ready to test through the full Alfred pipeline, enable prompt logging to see what the LLM actually receives and generates.

In [6]:
# Prompt logging is built into alfred-core.
# To enable, add to your .env:
#   ALFRED_LOG_PROMPTS=1      # Logs to prompt_logs/ directory (markdown files)
#   ALFRED_LOG_TO_DB=1        # Logs to Supabase prompt_logs table
#   ALFRED_LOG_KEEP_SESSIONS=4  # Auto-cleanup, keeps last 4 sessions
#
# After running Alfred, check prompt_logs/ for:
#   - System prompt sent to LLM at each step
#   - User prompt (includes persona, examples, data context)
#   - LLM response (parsed JSON)
#   - Token usage

from pathlib import Path

log_dir = Path('../prompt_logs')
if log_dir.exists():
    logs = sorted(log_dir.glob('*'), key=lambda p: p.stat().st_mtime, reverse=True)
    print(f'Found {len(logs)} prompt log files')
    if logs:
        latest = logs[0]
        print(f'Latest: {latest.name}')
        print(f'Size: {latest.stat().st_size / 1024:.1f} KB')
        # Show first 500 chars
        print(f'\nPreview:\n{latest.read_text()[:500]}...')
else:
    print('No prompt_logs/ directory found.')
    print('Enable with: ALFRED_LOG_PROMPTS=1 in .env')
    print('Then run Alfred and check prompt_logs/ for logged prompts.')

No prompt_logs/ directory found.
Enable with: ALFRED_LOG_PROMPTS=1 in .env
Then run Alfred and check prompt_logs/ for logged prompts.
